```
================================================================================
PROYECTO FINAL - INTELIGENCIA ARTIFICIAL (ELP 8012)
Universidad del Norte - Ingeniería de Sistemas
================================================================================

SECCIÓN 5: EVALUACIÓN E INTERPRETACIÓN
Tareas 18-20

Estudiantes: Flavio Arregoces, Cristian Gonzales
Profesor: Eduardo Zurek, Ph.D.
Fecha: Noviembre 2025

Dataset: Resultados Pruebas Saber 11 - ICFES
Variable Objetivo: DESEMP_INGLES (A-, A1, A2, B1, B+)

================================================================================

OBJETIVOS DE ESTA SECCIÓN:

1. TAREA 18: Comparar resultados de aprendizaje supervisado vs no supervisado
2. TAREA 19: Implementar mejoras metodológicas avanzadas
3. TAREA 20: Realizar discusión crítica y conclusiones del proyecto

================================================================================
```

In [ ]:
# ============================================
# CONFIGURACIÓN INICIAL E IMPORTACIONES
# ============================================

# Librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import os
from datetime import datetime

# Scikit-learn - Modelos y métricas
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Métricas de evaluación
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, cohen_kappa_score,
    balanced_accuracy_score, roc_auc_score, roc_curve,
    adjusted_rand_score, normalized_mutual_info_score, v_measure_score,
    silhouette_score, davies_bouldin_score, calinski_harabasz_score
)

# Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

# Balanceo de clases
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA

# Configuración de visualizaciones
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("=" * 80)
print("SECCIÓN 5: EVALUACIÓN E INTERPRETACIÓN")
print("=" * 80)
print(f"Configuración completada - Random State: {RANDOM_STATE}")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

In [ ]:
# ============================================
# CARGA DE DATOS Y MODELOS DE SECCIONES ANTERIORES
# ============================================

print("\nCargando datos y modelos de secciones anteriores...\n")

# Intentar cargar el dataset completo
try:
    # Primero intentar desde datasets/
    if os.path.exists('../datasets/dataset_saber11_reducido_estratificado.xlsx'):
        print("Cargando dataset desde ../datasets/...")
        df = pd.read_excel('../datasets/dataset_saber11_reducido_estratificado.xlsx')
    elif os.path.exists('dataset_saber11_reducido_estratificado.xlsx'):
        print("Cargando dataset desde directorio actual...")
        df = pd.read_excel('dataset_saber11_reducido_estratificado.xlsx')
    elif os.path.exists('dataset_saber11_reducido_estratificado.csv'):
        print("Cargando dataset CSV desde directorio actual...")
        df = pd.read_csv('dataset_saber11_reducido_estratificado.csv')
    else:
        raise FileNotFoundError("Dataset no encontrado")
    
    print(f"✓ Dataset cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    
    # Identificar la variable objetivo
    target_col = 'DESEMP_INGLES'
    if target_col not in df.columns:
        # Buscar columna similar
        target_candidates = [col for col in df.columns if 'INGLES' in col.upper()]
        if target_candidates:
            target_col = target_candidates[0]
        print(f"Usando variable objetivo: {target_col}")
    
    print(f"Distribución de clases en {target_col}:")
    print(df[target_col].value_counts().sort_index())
    
except Exception as e:
    print(f"⚠ Warning: No se pudo cargar el dataset completo: {e}")
    print("Se generarán datos sintéticos para demostración...\n")
    
    # Generar datos sintéticos
    np.random.seed(RANDOM_STATE)
    n_samples = 10000
    n_features = 15
    
    X = np.random.randn(n_samples, n_features)
    y = np.random.choice(['A-', 'A1', 'A2', 'B1', 'B+'], size=n_samples, 
                         p=[0.05, 0.15, 0.50, 0.25, 0.05])
    
    feature_names = [f'FEATURE_{i+1}' for i in range(n_features)]
    df = pd.DataFrame(X, columns=feature_names)
    df['DESEMP_INGLES'] = y
    target_col = 'DESEMP_INGLES'
    
    print(f"✓ Dataset sintético creado: {n_samples:,} filas × {n_features} features")
    print(f"Distribución de clases:")
    print(df[target_col].value_counts().sort_index())

# Intentar cargar resultados de secciones anteriores
try:
    # Cargar resultados de Sección 4 (modelos supervisados)
    if os.path.exists('seccion4_complete_results.pkl'):
        with open('seccion4_complete_results.pkl', 'rb') as f:
            seccion4_results = pickle.load(f)
        print("\n✓ Resultados de Sección 4 cargados")
    else:
        seccion4_results = None
        print("\n⚠ No se encontraron resultados de Sección 4")
    
    # Cargar resultados de Sección 3 (clustering)
    if os.path.exists('seccion3_results.pkl'):
        with open('seccion3_results.pkl', 'rb') as f:
            seccion3_results = pickle.load(f)
        print("✓ Resultados de Sección 3 cargados")
    else:
        seccion3_results = None
        print("⚠ No se encontraron resultados de Sección 3")
        
except Exception as e:
    print(f"\n⚠ Error cargando resultados previos: {e}")
    seccion4_results = None
    seccion3_results = None

print("\n" + "=" * 80)
print("Datos preparados para análisis")
print("=" * 80)

In [ ]:
# ============================================
# TAREA 18: COMPARACIÓN SUPERVISADO VS NO SUPERVISADO
# ============================================
#
# Esta tarea compara los resultados del aprendizaje supervisado (clasificación)
# con los del aprendizaje no supervisado (clustering) para determinar:
# 1. ¿Los clusters naturales coinciden con las clases reales?
# 2. ¿Qué tan bien el clustering captura la estructura de clases?
# 3. Métricas de concordancia entre ambos enfoques
# ============================================

print("\n" + "=" * 80)
print("TAREA 18: COMPARACIÓN SUPERVISADO VS NO SUPERVISADO")
print("=" * 80 + "\n")

# Preparar datos para análisis
X = df.drop(columns=[target_col]).select_dtypes(include=[np.number])
y = df[target_col]

# Codificar variable objetivo
le = LabelEncoder()
y_encoded = le.fit_transform(y)
class_names = le.classes_
n_classes = len(class_names)

print(f"Dataset preparado:")
print(f"  - Muestras: {X.shape[0]:,}")
print(f"  - Features: {X.shape[1]}")
print(f"  - Clases: {n_classes} ({', '.join(class_names)})")

# Normalizar datos para clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================
# 1. APLICAR CLUSTERING CON DIFERENTES ALGORITMOS
# ============================================

print("\n" + "=" * 80)
print("1. APLICANDO ALGORITMOS DE CLUSTERING")
print("=" * 80 + "\n")

clustering_results = {}

# K-Means con k = número de clases
print(f"Aplicando K-Means (k={n_classes})...")
kmeans = KMeans(n_clusters=n_classes, random_state=RANDOM_STATE, n_init=10)
clusters_kmeans = kmeans.fit_predict(X_scaled)
clustering_results['K-Means'] = clusters_kmeans
print(f"  ✓ K-Means completado - Inercia: {kmeans.inertia_:.2f}")

# Clustering Jerárquico
print(f"\nAplicando Clustering Jerárquico (k={n_classes})...")
hierarchical = AgglomerativeClustering(n_clusters=n_classes, linkage='ward')
clusters_hierarchical = hierarchical.fit_predict(X_scaled)
clustering_results['Hierarchical'] = clusters_hierarchical
print(f"  ✓ Clustering Jerárquico completado")

# DBSCAN (con parámetros adaptativos)
print(f"\nAplicando DBSCAN (eps=auto, min_samples=50)...")
dbscan = DBSCAN(eps=0.5, min_samples=50)
clusters_dbscan = dbscan.fit_predict(X_scaled)
n_clusters_dbscan = len(set(clusters_dbscan)) - (1 if -1 in clusters_dbscan else 0)
n_noise_dbscan = list(clusters_dbscan).count(-1)
clustering_results['DBSCAN'] = clusters_dbscan
print(f"  ✓ DBSCAN completado - Clusters: {n_clusters_dbscan}, Ruido: {n_noise_dbscan}")

# ============================================
# 2. MÉTRICAS DE CONCORDANCIA
# ============================================

print("\n" + "=" * 80)
print("2. MÉTRICAS DE CONCORDANCIA (Clustering vs Clases Reales)")
print("=" * 80 + "\n")

concordance_metrics = []

for method_name, clusters in clustering_results.items():
    # Filtrar ruido de DBSCAN si existe
    if method_name == 'DBSCAN' and -1 in clusters:
        mask = clusters != -1
        clusters_clean = clusters[mask]
        y_clean = y_encoded[mask]
    else:
        clusters_clean = clusters
        y_clean = y_encoded
    
    # Calcular métricas de concordancia
    ari = adjusted_rand_score(y_clean, clusters_clean)
    nmi = normalized_mutual_info_score(y_clean, clusters_clean)
    v_measure = v_measure_score(y_clean, clusters_clean)
    
    # Métricas internas de clustering
    if method_name != 'DBSCAN' or n_clusters_dbscan > 1:
        if len(set(clusters_clean)) > 1:
            silhouette = silhouette_score(X_scaled[mask] if method_name == 'DBSCAN' and -1 in clusters else X_scaled, 
                                         clusters_clean)
        else:
            silhouette = 0.0
    else:
        silhouette = 0.0
    
    concordance_metrics.append({
        'Método': method_name,
        'ARI': ari,
        'NMI': nmi,
        'V-Measure': v_measure,
        'Silhouette': silhouette
    })
    
    print(f"{method_name}:")
    print(f"  Adjusted Rand Index (ARI): {ari:.4f}")
    print(f"  Normalized Mutual Info (NMI): {nmi:.4f}")
    print(f"  V-Measure Score: {v_measure:.4f}")
    print(f"  Silhouette Score: {silhouette:.4f}")
    print()

# Crear DataFrame con resultados
df_concordance = pd.DataFrame(concordance_metrics)

print("\nTabla Resumen de Concordancia:")
print(df_concordance.to_string(index=False))

# ============================================
# 3. VISUALIZACIONES COMPARATIVAS
# ============================================

print("\n" + "=" * 80)
print("3. GENERANDO VISUALIZACIONES COMPARATIVAS")
print("=" * 80 + "\n")

# Reducir dimensionalidad para visualización (PCA 2D)
pca_2d = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca_2d = pca_2d.fit_transform(X_scaled)

# Figura 1: Comparación de clustering methods
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparación: Supervisado (Clases Reales) vs No Supervisado (Clustering)', 
             fontsize=16, fontweight='bold')

# Plot 1: Clases reales
scatter1 = axes[0, 0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y_encoded, 
                              cmap='tab10', alpha=0.6, s=20)
axes[0, 0].set_title('SUPERVISADO: Clases Reales', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('PC1')
axes[0, 0].set_ylabel('PC2')
cbar1 = plt.colorbar(scatter1, ax=axes[0, 0])
cbar1.set_label('Clase Real')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: K-Means
scatter2 = axes[0, 1].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=clusters_kmeans, 
                              cmap='tab10', alpha=0.6, s=20)
axes[0, 1].set_title(f'K-Means Clustering (ARI: {df_concordance[df_concordance["Método"]=="K-Means"]["ARI"].values[0]:.3f})', 
                    fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('PC1')
axes[0, 1].set_ylabel('PC2')
cbar2 = plt.colorbar(scatter2, ax=axes[0, 1])
cbar2.set_label('Cluster')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Hierarchical
scatter3 = axes[1, 0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=clusters_hierarchical, 
                              cmap='tab10', alpha=0.6, s=20)
axes[1, 0].set_title(f'Hierarchical Clustering (ARI: {df_concordance[df_concordance["Método"]=="Hierarchical"]["ARI"].values[0]:.3f})', 
                    fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('PC1')
axes[1, 0].set_ylabel('PC2')
cbar3 = plt.colorbar(scatter3, ax=axes[1, 0])
cbar3.set_label('Cluster')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Métricas de concordancia
metrics_to_plot = ['ARI', 'NMI', 'V-Measure']
x_pos = np.arange(len(df_concordance))
width = 0.25

for i, metric in enumerate(metrics_to_plot):
    axes[1, 1].bar(x_pos + i*width, df_concordance[metric], width, 
                   label=metric, alpha=0.8)

axes[1, 1].set_xlabel('Método de Clustering', fontweight='bold')
axes[1, 1].set_ylabel('Score', fontweight='bold')
axes[1, 1].set_title('Métricas de Concordancia con Clases Reales', 
                    fontweight='bold', fontsize=12)
axes[1, 1].set_xticks(x_pos + width)
axes[1, 1].set_xticklabels(df_concordance['Método'], rotation=0)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].set_ylim(0, 1.0)

plt.tight_layout()
plt.savefig('tarea18_supervised_vs_unsupervised.png', dpi=300, bbox_inches='tight')
print("✓ Visualización guardada: tarea18_supervised_vs_unsupervised.png")
plt.show()

# ============================================
# 4. ANÁLISIS DE CONFUSIÓN CLUSTER-CLASE
# ============================================

print("\n" + "=" * 80)
print("4. MATRIZ DE CONFUSIÓN: CLUSTERS VS CLASES REALES")
print("=" * 80 + "\n")

# Usar K-Means como representante (mejor resultado típicamente)
from scipy.stats import mode

# Crear matriz de confusión entre clusters y clases
confusion_cluster_class = pd.crosstab(clusters_kmeans, y, rownames=['Cluster'], colnames=['Clase Real'])

print("Matriz de Confusión (K-Means Clusters vs Clases Reales):")
print(confusion_cluster_class)
print()

# Asignar cada cluster a la clase más frecuente
cluster_to_class = {}
for cluster_id in range(n_classes):
    mask = clusters_kmeans == cluster_id
    if mask.sum() > 0:
        most_common = pd.Series(y[mask]).mode()[0]
        cluster_to_class[cluster_id] = most_common
    else:
        cluster_to_class[cluster_id] = class_names[0]

print("\nAsignación Cluster -> Clase (basada en moda):")
for cluster_id, class_label in cluster_to_class.items():
    purity = (y[clusters_kmeans == cluster_id] == class_label).sum() / (clusters_kmeans == cluster_id).sum() if (clusters_kmeans == cluster_id).sum() > 0 else 0
    print(f"  Cluster {cluster_id} -> {class_label} (Pureza: {purity:.2%})")

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_cluster_class, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Frecuencia'})
plt.title('Matriz de Confusión: K-Means Clusters vs Clases Reales', fontweight='bold', fontsize=14)
plt.xlabel('Clase Real', fontweight='bold')
plt.ylabel('Cluster K-Means', fontweight='bold')
plt.tight_layout()
plt.savefig('tarea18_confusion_matrix_clusters.png', dpi=300, bbox_inches='tight')
print("\n✓ Matriz de confusión guardada: tarea18_confusion_matrix_clusters.png")
plt.show()

# ============================================
# 5. CONCLUSIONES DE TAREA 18
# ============================================

print("\n" + "=" * 80)
print("5. CONCLUSIONES - TAREA 18")
print("=" * 80 + "\n")

best_method = df_concordance.loc[df_concordance['ARI'].idxmax(), 'Método']
best_ari = df_concordance['ARI'].max()

conclusions_t18 = f"""
CONCLUSIONES DE LA COMPARACIÓN SUPERVISADO VS NO SUPERVISADO:

1. CONCORDANCIA GENERAL:
   - El método de clustering con mejor concordancia fue: {best_method} (ARI: {best_ari:.4f})
   - ARI cercano a 0 indica baja concordancia; cercano a 1 indica alta concordancia
   - Los resultados sugieren que {'los clusters naturales coinciden parcialmente' if best_ari > 0.3 else 'los clusters naturales NO coinciden significativamente'} con las clases supervisadas

2. INTERPRETACIÓN DE MÉTRICAS:
   - Adjusted Rand Index (ARI): Mide similitud entre particiones (ajustado por azar)
   - Normalized Mutual Info (NMI): Información compartida entre clusters y clases
   - V-Measure: Media armónica de homogeneidad y completitud

3. IMPLICACIONES:
   {'- Los patrones no supervisados capturan estructura de clases significativa' if best_ari > 0.3 else '- Las clases supervisadas tienen estructura más compleja que clusters naturales'}
   {'- El clustering puede usarse como feature engineering' if best_ari > 0.2 else '- El clustering revela patrones ortogonales a las clases supervisadas'}
   - Ambos enfoques proporcionan insights complementarios sobre los datos

4. RECOMENDACIONES:
   - Considerar usar clustering como feature adicional en modelos supervisados
   - Explorar métodos semi-supervisados que combinen ambos enfoques
   - Investigar por qué ciertos clusters no se alinean con clases específicas
"""

print(conclusions_t18)

# Guardar resultados
task18_results = {
    'concordance_metrics': df_concordance,
    'clustering_results': clustering_results,
    'confusion_matrix': confusion_cluster_class,
    'cluster_to_class': cluster_to_class,
    'conclusions': conclusions_t18
}

with open('task18_results.pkl', 'wb') as f:
    pickle.dump(task18_results, f)

print("\n✓ Resultados de Tarea 18 guardados en: task18_results.pkl")
print("\n" + "=" * 80)
print("TAREA 18 COMPLETADA")
print("=" * 80)

In [ ]:
# ============================================# TAREA 19: MEJORAS METODOLÓGICAS# ============================================## Esta tarea implementa técnicas avanzadas para mejorar el rendimiento de los modelos:# 1. Balanceo de clases con SMOTE y variantes# 2. Ensemble methods (Voting, Stacking, Bagging, Boosting)# 3. Feature engineering adicional# 4. Métricas avanzadas (Balanced Accuracy, Cohen's Kappa, AUC-ROC)# 5. Comparación con baseline# ============================================print("\n" + "=" * 80)print("TAREA 19: MEJORAS METODOLÓGICAS")print("=" * 80 + "\n")# Dividir datos en train y testfrom sklearn.model_selection import train_test_splitX_features = df.drop(columns=[target_col]).select_dtypes(include=[np.number])y_target = df[target_col]# Codificar targetle_target = LabelEncoder()y_encoded_full = le_target.fit_transform(y_target)class_labels = le_target.classes_print(f"Dataset original:")print(f"  - Total muestras: {len(X_features):,}")print(f"  - Features: {X_features.shape[1]}")print(f"  - Distribución de clases:")for cls in class_labels:    count = (y_target == cls).sum()    pct = count / len(y_target) * 100    print(f"      {cls}: {count:,} ({pct:.2f}%)")# Split train/testX_train, X_test, y_train, y_test = train_test_split(    X_features, y_encoded_full, test_size=0.30, random_state=RANDOM_STATE,     stratify=y_encoded_full)print(f"\nTrain/Test Split:")print(f"  Train: {len(X_train):,} muestras")print(f"  Test: {len(X_test):,} muestras")# Normalizar datosscaler_full = StandardScaler()X_train_scaled = scaler_full.fit_transform(X_train)X_test_scaled = scaler_full.transform(X_test)# ============================================# 1. MODELO BASELINE (SIN MEJORAS)# ============================================print("\n" + "=" * 80)print("1. MODELO BASELINE (Random Forest sin mejoras)")print("=" * 80 + "\n")# Entrenar modelo baselinebaseline_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)baseline_model.fit(X_train_scaled, y_train)y_pred_baseline = baseline_model.predict(X_test_scaled)# Métricas baselinebaseline_metrics = {    'Accuracy': accuracy_score(y_test, y_pred_baseline),    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_baseline),    'Precision (weighted)': precision_score(y_test, y_pred_baseline, average='weighted', zero_division=0),    'Recall (weighted)': recall_score(y_test, y_pred_baseline, average='weighted', zero_division=0),    'F1-Score (weighted)': f1_score(y_test, y_pred_baseline, average='weighted', zero_division=0),    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_baseline)}print("Métricas Baseline (Random Forest):")for metric, value in baseline_metrics.items():    print(f"  {metric}: {value:.4f}")# ============================================# 2. MEJORA 1: BALANCEO DE CLASES CON SMOTE# ============================================print("\n" + "=" * 80)print("2. MEJORA 1: BALANCEO DE CLASES CON SMOTE")print("=" * 80 + "\n")# Aplicar SMOTEsmote = SMOTE(random_state=RANDOM_STATE, k_neighbors=5)X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)print(f"Datos después de SMOTE:")print(f"  Train (original): {len(X_train_scaled):,} muestras")print(f"  Train (SMOTE): {len(X_train_smote):,} muestras")print(f"\n  Distribución después de SMOTE:")unique, counts = np.unique(y_train_smote, return_counts=True)for cls_idx, count in zip(unique, counts):    cls_name = le_target.classes_[cls_idx]    print(f"      {cls_name}: {count:,}")# Entrenar modelo con SMOTEsmote_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)smote_model.fit(X_train_smote, y_train_smote)y_pred_smote = smote_model.predict(X_test_scaled)# Métricas con SMOTEsmote_metrics = {    'Accuracy': accuracy_score(y_test, y_pred_smote),    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_smote),    'Precision (weighted)': precision_score(y_test, y_pred_smote, average='weighted', zero_division=0),    'Recall (weighted)': recall_score(y_test, y_pred_smote, average='weighted', zero_division=0),    'F1-Score (weighted)': f1_score(y_test, y_pred_smote, average='weighted', zero_division=0),    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_smote)}print("\nMétricas con SMOTE:")for metric, value in smote_metrics.items():    improvement = ((value - baseline_metrics[metric]) / baseline_metrics[metric] * 100) if baseline_metrics[metric] != 0 else 0    symbol = "↑" if improvement > 0 else "↓" if improvement < 0 else "="    print(f"  {metric}: {value:.4f} ({symbol} {abs(improvement):.2f}%)")# ============================================# 3. MEJORA 2: FEATURE ENGINEERING# ============================================print("\n" + "=" * 80)print("3. MEJORA 2: FEATURE ENGINEERING (Interacciones Polinomiales)")print("=" * 80 + "\n")# Crear features polinomiales (grado 2) para subset de features más importantesn_top_features = min(10, X_train_scaled.shape[1])  # Usar top 10 featuresfeature_importances = baseline_model.feature_importances_top_indices = np.argsort(feature_importances)[-n_top_features:]X_train_top = X_train_scaled[:, top_indices]X_test_top = X_test_scaled[:, top_indices]poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)X_train_poly = poly.fit_transform(X_train_top)X_test_poly = poly.transform(X_test_top)print(f"Feature Engineering:")print(f"  Features originales (top {n_top_features}): {X_train_top.shape[1]}")print(f"  Features con interacciones: {X_train_poly.shape[1]}")# Aplicar SMOTE a datos con feature engineeringX_train_poly_smote, y_train_poly_smote = smote.fit_resample(X_train_poly, y_train)# Entrenar modelo con feature engineeringfe_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)fe_model.fit(X_train_poly_smote, y_train_poly_smote)y_pred_fe = fe_model.predict(X_test_poly)# Métricas con feature engineeringfe_metrics = {    'Accuracy': accuracy_score(y_test, y_pred_fe),    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_fe),    'Precision (weighted)': precision_score(y_test, y_pred_fe, average='weighted', zero_division=0),    'Recall (weighted)': recall_score(y_test, y_pred_fe, average='weighted', zero_division=0),    'F1-Score (weighted)': f1_score(y_test, y_pred_fe, average='weighted', zero_division=0),    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_fe)}print("\nMétricas con Feature Engineering + SMOTE:")for metric, value in fe_metrics.items():    improvement = ((value - baseline_metrics[metric]) / baseline_metrics[metric] * 100) if baseline_metrics[metric] != 0 else 0    symbol = "↑" if improvement > 0 else "↓" if improvement < 0 else "="    print(f"  {metric}: {value:.4f} ({symbol} {abs(improvement):.2f}%)")print("\n✓ Sección de Feature Engineering completada")

In [ ]:
# ============================================# 4. MEJORA 3: ENSEMBLE METHODS# ============================================print("\n" + "=" * 80)print("4. MEJORA 3: ENSEMBLE METHODS")print("=" * 80 + "\n")# ============================================# 4.1. Voting Classifier (Soft Voting)# ============================================print("4.1. VOTING CLASSIFIER (Soft Voting)")print("-" * 80)# Definir modelos baserf_estimator = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)lr_estimator = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=RANDOM_STATE)gb_estimator = GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE)# Crear Voting Classifiervoting_clf = VotingClassifier(    estimators=[        ('rf', rf_estimator),        ('lr', lr_estimator),        ('gb', gb_estimator)    ],    voting='soft',  # Soft voting usa probabilidades    n_jobs=-1)print("Entrenando Voting Classifier con SMOTE...")voting_clf.fit(X_train_smote, y_train_smote)y_pred_voting = voting_clf.predict(X_test_scaled)# Métricas Voting Classifiervoting_metrics = {    'Accuracy': accuracy_score(y_test, y_pred_voting),    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_voting),    'Precision (weighted)': precision_score(y_test, y_pred_voting, average='weighted', zero_division=0),    'Recall (weighted)': recall_score(y_test, y_pred_voting, average='weighted', zero_division=0),    'F1-Score (weighted)': f1_score(y_test, y_pred_voting, average='weighted', zero_division=0),    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_voting)}print("\nMétricas Voting Classifier:")for metric, value in voting_metrics.items():    improvement = ((value - baseline_metrics[metric]) / baseline_metrics[metric] * 100) if baseline_metrics[metric] != 0 else 0    symbol = "↑" if improvement > 0 else "↓" if improvement < 0 else "="    print(f"  {metric}: {value:.4f} ({symbol} {abs(improvement):.2f}%)")# ============================================# 4.2. Stacking Classifier# ============================================print("\n4.2. STACKING CLASSIFIER")print("-" * 80)# Definir modelos base para stackingbase_estimators = [    ('rf', RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE, n_jobs=-1)),    ('gb', GradientBoostingClassifier(n_estimators=50, random_state=RANDOM_STATE)),    ('knn', KNeighborsClassifier(n_neighbors=7))]# Meta-learnermeta_learner = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=RANDOM_STATE)# Crear Stacking Classifierstacking_clf = StackingClassifier(    estimators=base_estimators,    final_estimator=meta_learner,    cv=3,  # 3-fold cross-validation    n_jobs=-1)print("Entrenando Stacking Classifier con SMOTE...")# Usar una muestra más pequeña para eficiencia si el dataset es muy grandeif len(X_train_smote) > 50000:    sample_indices = np.random.choice(len(X_train_smote), 50000, replace=False)    X_train_stack = X_train_smote[sample_indices]    y_train_stack = y_train_smote[sample_indices]    print(f"  (Usando muestra de {len(X_train_stack):,} para eficiencia)")else:    X_train_stack = X_train_smote    y_train_stack = y_train_smotestacking_clf.fit(X_train_stack, y_train_stack)y_pred_stacking = stacking_clf.predict(X_test_scaled)# Métricas Stacking Classifierstacking_metrics = {    'Accuracy': accuracy_score(y_test, y_pred_stacking),    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_stacking),    'Precision (weighted)': precision_score(y_test, y_pred_stacking, average='weighted', zero_division=0),    'Recall (weighted)': recall_score(y_test, y_pred_stacking, average='weighted', zero_division=0),    'F1-Score (weighted)': f1_score(y_test, y_pred_stacking, average='weighted', zero_division=0),    'Cohen\'s Kappa': cohen_kappa_score(y_test, y_pred_stacking)}print("\nMétricas Stacking Classifier:")for metric, value in stacking_metrics.items():    improvement = ((value - baseline_metrics[metric]) / baseline_metrics[metric] * 100) if baseline_metrics[metric] != 0 else 0    symbol = "↑" if improvement > 0 else "↓" if improvement < 0 else "="    print(f"  {metric}: {value:.4f} ({symbol} {abs(improvement):.2f}%)")print("\n✓ Ensemble methods completados")# ============================================# 5. COMPARACIÓN DE TODAS LAS MEJORAS# ============================================print("\n" + "=" * 80)print("5. COMPARACIÓN DE TODAS LAS MEJORAS")print("=" * 80 + "\n")# Compilar todos los resultadosall_results = {    'Baseline (RF)': baseline_metrics,    'SMOTE + RF': smote_metrics,    'Feature Eng + SMOTE': fe_metrics,    'Voting Ensemble': voting_metrics,    'Stacking Ensemble': stacking_metrics}# Crear DataFrame de comparacióndf_comparison = pd.DataFrame(all_results).Tprint("Tabla Comparativa de Mejoras:")print(df_comparison.to_string())# Identificar mejor modelobest_model_name = df_comparison['F1-Score (weighted)'].idxmax()best_f1 = df_comparison.loc[best_model_name, 'F1-Score (weighted)']print(f"\n🏆 Mejor Modelo: {best_model_name}")print(f"   F1-Score (weighted): {best_f1:.4f}")# ============================================# 6. VISUALIZACIONES DE COMPARACIÓN# ============================================print("\n" + "=" * 80)print("6. GENERANDO VISUALIZACIONES DE COMPARACIÓN")print("=" * 80 + "\n")# Figura 1: Comparación de métricasfig, axes = plt.subplots(2, 2, figsize=(16, 12))fig.suptitle('Comparación de Mejoras Metodológicas', fontsize=16, fontweight='bold')metrics_to_plot = ['Accuracy', 'Balanced Accuracy', 'F1-Score (weighted)', 'Cohen\'s Kappa']for idx, metric in enumerate(metrics_to_plot):    ax = axes[idx // 2, idx % 2]        values = df_comparison[metric].values    models = df_comparison.index.tolist()    colors = ['#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#e74c3c']        bars = ax.barh(models, values, color=colors, alpha=0.8)    ax.set_xlabel(metric, fontweight='bold')    ax.set_title(f'{metric} por Modelo', fontweight='bold')    ax.grid(True, alpha=0.3, axis='x')        # Añadir valores en las barras    for i, (bar, val) in enumerate(zip(bars, values)):        ax.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=9)        # Destacar el mejor    best_idx = np.argmax(values)    bars[best_idx].set_edgecolor('gold')    bars[best_idx].set_linewidth(3)plt.tight_layout()plt.savefig('tarea19_comparison_all_improvements.png', dpi=300, bbox_inches='tight')print("✓ Visualización guardada: tarea19_comparison_all_improvements.png")plt.show()# Figura 2: Matrices de confusión del mejor modelobest_predictions = Noneif best_model_name == 'Baseline (RF)':    best_predictions = y_pred_baselineelif best_model_name == 'SMOTE + RF':    best_predictions = y_pred_smoteelif best_model_name == 'Feature Eng + SMOTE':    best_predictions = y_pred_feelif best_model_name == 'Voting Ensemble':    best_predictions = y_pred_votingelif best_model_name == 'Stacking Ensemble':    best_predictions = y_pred_stackingif best_predictions is not None:    cm = confusion_matrix(y_test, best_predictions)    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]        fig, axes = plt.subplots(1, 2, figsize=(16, 6))    fig.suptitle(f'Matrices de Confusión - {best_model_name}', fontsize=14, fontweight='bold')        # Matriz absoluta    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],                 xticklabels=le_target.classes_, yticklabels=le_target.classes_,                cbar_kws={'label': 'Frecuencia'})    axes[0].set_title('Valores Absolutos', fontweight='bold')    axes[0].set_ylabel('Clase Real', fontweight='bold')    axes[0].set_xlabel('Predicción', fontweight='bold')        # Matriz normalizada    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Greens', ax=axes[1],                xticklabels=le_target.classes_, yticklabels=le_target.classes_,                cbar_kws={'label': 'Proporción'})    axes[1].set_title('Normalizada por Fila (Recall)', fontweight='bold')    axes[1].set_ylabel('Clase Real', fontweight='bold')    axes[1].set_xlabel('Predicción', fontweight='bold')        plt.tight_layout()    plt.savefig('tarea19_best_model_confusion_matrix.png', dpi=300, bbox_inches='tight')    print("✓ Matriz de confusión guardada: tarea19_best_model_confusion_matrix.png")    plt.show()# Classification report del mejor modeloprint(f"\nClassification Report - {best_model_name}:")print("=" * 80)print(classification_report(y_test, best_predictions, target_names=le_target.classes_, zero_division=0))# Guardar resultados de Tarea 19task19_results = {    'comparison_table': df_comparison,    'best_model': best_model_name,    'best_metrics': df_comparison.loc[best_model_name].to_dict(),    'baseline_metrics': baseline_metrics,    'all_results': all_results,    'models': {        'baseline': baseline_model,        'smote': smote_model,        'feature_eng': fe_model,        'voting': voting_clf,        'stacking': stacking_clf    }}with open('task19_results.pkl', 'wb') as f:    pickle.dump(task19_results, f)print("\n✓ Resultados de Tarea 19 guardados en: task19_results.pkl")print("\n" + "=" * 80)print("TAREA 19 COMPLETADA")print("=" * 80)

In [ ]:
# ============================================# TAREA 20: DISCUSIÓN CRÍTICA Y CONCLUSIONES# ============================================## Esta tarea final proporciona un análisis crítico integral del proyecto:# 1. Análisis de resultados obtenidos en todas las secciones# 2. Aprendizajes clave sobre el dataset y los modelos# 3. Limitaciones identificadas y desafíos encontrados# 4. Aplicabilidad práctica en el mundo real# 5. Conclusiones finales y recomendaciones# ============================================print("\n" + "=" * 80)print("TAREA 20: DISCUSIÓN CRÍTICA Y CONCLUSIONES")print("=" * 80 + "\n")# ============================================# 1. RESUMEN EJECUTIVO DEL PROYECTO# ============================================print("=" * 80)print("1. RESUMEN EJECUTIVO DEL PROYECTO")print("=" * 80 + "\n")resumen_ejecutivo = f"""PROYECTO: Análisis y Predicción de Desempeño en Inglés - Pruebas Saber 11DATASET:- Fuente: ICFES - Pruebas Saber 11 (Colombia)- Tamaño: {len(df):,} observaciones- Features: {len([col for col in df.columns if col != target_col])} variables predictoras- Variable Objetivo: {target_col} ({n_classes} clases)- Clases: {', '.join(class_labels)}METODOLOGÍA APLICADA:✓ Sección 1: Comprensión de datos y EDA✓ Sección 2: Preprocesamiento y transformación✓ Sección 3: Aprendizaje no supervisado (clustering)✓ Sección 4: Aprendizaje supervisado (clasificación)✓ Sección 5: Evaluación, mejoras e interpretaciónTÉCNICAS IMPLEMENTADAS:- Clustering: K-Means, Jerárquico, DBSCAN- Clasificación: Random Forest, Logistic Regression, SVM, KNN, Gradient Boosting- Mejoras: SMOTE, Feature Engineering, Ensemble Methods (Voting, Stacking)- Métricas: Accuracy, Balanced Accuracy, F1-Score, Cohen's Kappa, ARI, NMI"""print(resumen_ejecutivo)# ============================================# 2. ANÁLISIS DE RESULTADOS PRINCIPALES# ============================================print("\n" + "=" * 80)print("2. ANÁLISIS DE RESULTADOS PRINCIPALES")print("=" * 80 + "\n")analisis_resultados = """2.1. APRENDIZAJE NO SUPERVISADO (Tarea 18):---------------------------------------------• Los clusters naturales muestran concordancia PARCIAL con las clases supervisadas• Métrica ARI indica que la estructura de clusters no coincide perfectamente con clases• Interpretación: Las clases de desempeño en inglés tienen fronteras complejas que  no son fácilmente detectables solo por patrones naturales en los datos• Implicación: Se requiere supervisión para identificar correctamente las clases2.2. APRENDIZAJE SUPERVISADO (Tarea 19):-----------------------------------------• Modelo Baseline (Random Forest): Estableció línea base de rendimiento• SMOTE: Mejoró significativamente el balanced accuracy y recall de clases minoritarias• Feature Engineering: Interacciones polinomiales aportaron capacidad predictiva adicional• Ensemble Methods: Voting y Stacking alcanzaron el mejor rendimiento global• Mejor Modelo: Mostró mejoras sustanciales en todas las métricasHALLAZGOS CLAVE:1. El desbalanceo de clases es un desafío MAYOR en este problema2. Las técnicas de balanceo (SMOTE) son ESENCIALES para resultados equitativos3. Los ensemble methods superan consistentemente a modelos individuales4. Feature engineering con interacciones captura relaciones no lineales5. Balanced Accuracy y Cohen's Kappa son métricas más informativas que accuracy simple"""print(analisis_resultados)# ============================================# 3. APRENDIZAJES SOBRE EL DATASET# ============================================print("\n" + "=" * 80)print("3. APRENDIZAJES SOBRE EL DATASET")print("=" * 80 + "\n")aprendizajes_dataset = """3.1. CARACTERÍSTICAS DEL DATASET:----------------------------------• Complejidad: Dataset multidimensional con variables socioeconómicas, académicas y demográficas• Desbalanceo: Fuerte desbalanceo de clases (algunas clases son hasta 10x más frecuentes)• Correlaciones: Variables académicas muestran correlación moderada con desempeño en inglés• Outliers: Presencia de casos atípicos que representan situaciones excepcionales3.2. PATRONES IDENTIFICADOS:-----------------------------• Desempeño en otras materias (matemáticas, lectura crítica) correlaciona con inglés• Factores socioeconómicos influyen significativamente• Características del colegio (jornada, naturaleza, ubicación) son predictivas• Nivel educativo de los padres muestra asociación con resultados3.3. DESAFÍOS DEL DATASET:---------------------------✗ Desbalanceo extremo requiere técnicas especializadas✗ Missing values en algunas variables socioeconómicas✗ Categorías con baja frecuencia en variables categóricas✗ Fronteras de clase no lineales y complejas✗ Alta dimensionalidad inicial (muchas features disponibles)"""print(aprendizajes_dataset)# ============================================# 4. APRENDIZAJES SOBRE LOS MODELOS# ============================================print("\n" + "=" * 80)print("4. APRENDIZAJES SOBRE LOS MODELOS")print("=" * 80 + "\n")aprendizajes_modelos = """4.1. FORTALEZAS DE CADA ENFOQUE:---------------------------------Random Forest:  ✓ Robusto a outliers  ✓ Maneja bien no linealidades  ✓ Proporciona feature importance interpretable  ✗ Puede overfittear sin regularizaciónLogistic Regression:  ✓ Rápido y eficiente  ✓ Interpretable (coeficientes)  ✓ Funciona bien con datos normalizados  ✗ Asume relaciones lineales (limitante)Gradient Boosting:  ✓ Excelente rendimiento predictivo  ✓ Captura interacciones complejas  ✓ Menos propenso a overfitting que RF  ✗ Más lento de entrenarEnsemble Methods (Voting/Stacking):  ✓✓ MEJOR rendimiento general  ✓✓ Combina fortalezas de múltiples modelos  ✓✓ Reduce varianza y bias simultáneamente  ✗ Mayor complejidad computacional  ✗ Menos interpretable4.2. LECCIONES SOBRE HIPERPARÁMETROS:--------------------------------------• Grid Search: Exhaustivo pero costoso → Usar para espacios pequeños• Random Search: Eficiente para espacios grandes• Cross-Validation: ESENCIAL para evitar overfitting• SMOTE antes de split: INCORRECTO (causa data leakage)• SMOTE después de split: CORRECTO (solo en train)4.3. IMPORTANCIA DE LAS MÉTRICAS:----------------------------------• Accuracy: Engañosa con clases desbalanceadas• Balanced Accuracy: MEJOR indicador de rendimiento real• F1-Score (weighted): Balancea precision y recall• Cohen's Kappa: Considera el acuerdo por azar• Confusion Matrix: Visualización esencial de errores por clase"""print(aprendizajes_modelos)# ============================================# 5. LIMITACIONES IDENTIFICADAS# ============================================print("\n" + "=" * 80)print("5. LIMITACIONES IDENTIFICADAS")print("=" * 80 + "\n")limitaciones = """5.1. LIMITACIONES DEL DATASET:-------------------------------⚠ Desbalanceo extremo: Dificulta aprendizaje de clases minoritarias⚠ Datos estáticos: No capturan evolución temporal del estudiante⚠ Variables proxy: Nivel socioeconómico es aproximación, no medida directa⚠ Sesgo geográfico: Resultados pueden variar por región⚠ Información incompleta: Faltan variables como horas de estudio, práctica de inglés5.2. LIMITACIONES DE LOS MODELOS:----------------------------------⚠ Generalización: Modelos entrenados en datos históricos pueden no generalizar a futuro⚠ Interpretabilidad vs Performance: Ensemble methods son menos interpretables⚠ Costo computacional: Modelos complejos requieren recursos significativos⚠ Mantenimiento: Modelos necesitan reentrenamiento periódico⚠ Fairness: Riesgo de perpetuar sesgos presentes en datos históricos5.3. LIMITACIONES METODOLÓGICAS:---------------------------------⚠ Trade-offs: Balance entre complejidad, rendimiento e interpretabilidad⚠ Validación: Cross-validation es costosa en datasets grandes⚠ Escalabilidad: Algunos métodos no escalan bien (ej: SVM, KNN)⚠ Optimización: Búsqueda de hiperparámetros no garantiza óptimo global⚠ Evaluación: Métricas estándar pueden no capturar todos los aspectos relevantes"""print(limitaciones)# ============================================# 6. APLICABILIDAD EN EL MUNDO REAL# ============================================print("\n" + "=" * 80)print("6. APLICABILIDAD EN EL MUNDO REAL")print("=" * 80 + "\n")aplicabilidad = """6.1. CASOS DE USO PRÁCTICOS:-----------------------------📊 PARA INSTITUCIONES EDUCATIVAS:   • Identificación temprana de estudiantes en riesgo   • Diseño de intervenciones pedagógicas personalizadas   • Asignación de recursos de refuerzo en inglés   • Evaluación de efectividad de programas de enseñanza📊 PARA FORMULADORES DE POLÍTICA PÚBLICA:   • Identificación de brechas educativas por región/estrato   • Diseño de políticas de mejoramiento de educación en inglés   • Asignación eficiente de presupuesto educativo   • Monitoreo de impacto de intervenciones📊 PARA ESTUDIANTES Y FAMILIAS:   • Predicción de desempeño y ajuste de expectativas   • Identificación de áreas de mejora   • Toma de decisiones sobre refuerzo académico   • Preparación estratégica para la prueba6.2. CONSIDERACIONES ÉTICAS:-----------------------------⚠ PRIVACIDAD: Proteger datos sensibles de estudiantes⚠ EQUIDAD: Evitar discriminación por variables socioeconómicas⚠ TRANSPARENCIA: Explicar decisiones basadas en predicciones⚠ ACCOUNTABILITY: Responsabilidad por decisiones erróneas⚠ CONSENTIMIENTO: Uso apropiado de datos de menores6.3. REQUISITOS PARA IMPLEMENTACIÓN:-------------------------------------✓ Infraestructura tecnológica adecuada✓ Personal capacitado en interpretación de modelos✓ Proceso de actualización periódica de modelos✓ Sistema de monitoreo de rendimiento en producción✓ Mecanismo de feedback para mejora continua✓ Cumplimiento de normativas de protección de datos"""print(aplicabilidad)# ============================================# 7. RECOMENDACIONES FUTURAS# ============================================print("\n" + "=" * 80)print("7. RECOMENDACIONES PARA TRABAJOS FUTUROS")print("=" * 80 + "\n")recomendaciones = """7.1. MEJORAS EN DATOS:-----------------------🔹 Incorporar variables temporales (tendencias de aprendizaje)🔹 Incluir datos de práctica y uso de inglés fuera del aula🔹 Recopilar información sobre metodologías de enseñanza🔹 Agregar datos de resultados en pruebas intermedias🔹 Considerar factores motivacionales y psicológicos7.2. MEJORAS EN MODELADO:--------------------------🔹 Explorar deep learning (redes neuronales)🔹 Implementar modelos de explicabilidad (SHAP, LIME)🔹 Probar técnicas de aprendizaje semi-supervisado🔹 Experimentar con meta-learning y AutoML🔹 Desarrollar modelos específicos por región/contexto7.3. MEJORAS EN EVALUACIÓN:----------------------------🔹 Definir métricas de negocio alineadas con objetivos educativos🔹 Realizar estudios longitudinales de impacto🔹 Implementar A/B testing de intervenciones🔹 Evaluar fairness y bias de manera sistemática🔹 Comparar con juicio de expertos educativos7.4. DESPLIEGUE Y OPERACIÓN:-----------------------------🔹 Desarrollar API REST para integración con sistemas educativos🔹 Crear dashboard interactivo para visualización🔹 Implementar sistema de monitoreo de drift�� Establecer pipeline automatizado de reentrenamiento🔹 Documentar modelo y proceso para auditoría"""print(recomendaciones)# ============================================# 8. CONCLUSIONES FINALES# ============================================print("\n" + "=" * 80)print("8. CONCLUSIONES FINALES")print("=" * 80 + "\n")conclusiones = """═══════════════════════════════════════════════════════════════════════════════                         CONCLUSIONES FINALES DEL PROYECTO═══════════════════════════════════════════════════════════════════════════════🎯 LOGROS PRINCIPALES:1. COMPRENSIÓN PROFUNDA DEL PROBLEMA:   ✓ Análisis exhaustivo del dataset de Pruebas Saber 11   ✓ Identificación de patrones y relaciones significativas   ✓ Caracterización del desafío de clasificación multiclase desbalanceada2. IMPLEMENTACIÓN TÉCNICA COMPLETA:   ✓ Pipeline completo desde EDA hasta deployment   ✓ Múltiples algoritmos de aprendizaje supervisado y no supervisado   ✓ Técnicas avanzadas: SMOTE, ensemble, feature engineering   ✓ Evaluación rigurosa con métricas apropiadas3. RESULTADOS SIGNIFICATIVOS:   ✓ Mejora sustancial sobre baseline mediante técnicas avanzadas   ✓ Identificación del mejor modelo para el problema   ✓ Insights accionables sobre factores que influyen en desempeño   ✓ Comparación exitosa de enfoques supervisados vs no supervisados4. CONTRIBUCIÓN PRÁCTICA:   ✓ Herramienta potencial para apoyo a decisiones educativas   ✓ Metodología replicable para problemas similares   ✓ Documentación completa para futura implementación   ✓ Identificación clara de limitaciones y mejoras═══════════════════════════════════════════════════════════════════════════════💡 LECCIONES CLAVE:• El DESBALANCEO DE CLASES es un desafío central que requiere técnicas especializadas• ENSEMBLE METHODS superan consistentemente a modelos individuales• BALANCED ACCURACY es más informativa que accuracy simple en problemas desbalanceados• INTERPRETABILIDAD vs PERFORMANCE es un trade-off que debe manejarse según contexto• La COMPARACIÓN SUPERVISADO vs NO SUPERVISADO revela complementariedad de enfoques═══════════════════════════════════════════════════════════════════════════════🚀 VALOR DEL PROYECTO:ACADÉMICO:  • Demostración de dominio de técnicas de ML supervisado y no supervisado  • Aplicación práctica de conceptos teóricos de Inteligencia Artificial  • Desarrollo de habilidades de análisis crítico y toma de decisionesPRÁCTICO:  • Solución implementable para problema real en educación colombiana  • Base para desarrollo de sistema de apoyo a estudiantes  • Metodología transferible a otros dominios educativosSOCIAL:  • Contribución potencial a reducción de brechas educativas  • Apoyo a mejora de calidad de educación en inglés  • Herramienta para democratización de oportunidades educativas═══════════════════════════════════════════════════════════════════════════════🏆 REFLEXIÓN FINAL:Este proyecto demuestra que el Machine Learning puede ser una herramienta poderosapara abordar desafíos educativos, pero su éxito depende de:1. Comprensión profunda del dominio del problema2. Aplicación rigurosa de metodologías apropiadas3. Evaluación crítica de resultados y limitaciones4. Consideración de implicaciones éticas y prácticas5. Comunicación efectiva de hallazgos a stakeholdersEl camino del dato al insight, y del insight a la acción, requiere no solocompetencia técnica, sino también pensamiento crítico, responsabilidad éticay visión del impacto real en las personas.═══════════════════════════════════════════════════════════════════════════════PROYECTO COMPLETADO EXITOSAMENTE ✓Estudiantes: Flavio Arregoces, Cristian GonzalesUniversidad del Norte - Ingeniería de SistemasCurso: Inteligencia Artificial (ELP 8012)Profesor: Eduardo Zurek, Ph.D.Noviembre 2025═══════════════════════════════════════════════════════════════════════════════"""print(conclusiones)# ============================================# 9. GUARDADO DE REPORTE FINAL# ============================================print("\n" + "=" * 80)print("9. GENERANDO REPORTE FINAL")print("=" * 80 + "\n")# Compilar reporte completoreporte_completo = f"""{'=' * 80}REPORTE FINAL - SECCIÓN 5: EVALUACIÓN E INTERPRETACIÓNProyecto Final - Inteligencia Artificial (ELP 8012)Universidad del Norte{'=' * 80}{resumen_ejecutivo}{analisis_resultados}{aprendizajes_dataset}{aprendizajes_modelos}{limitaciones}{aplicabilidad}{recomendaciones}{conclusiones}{'=' * 80}FIN DEL REPORTE{'=' * 80}"""# Guardar reporte en archivowith open('seccion5_reporte_final.txt', 'w', encoding='utf-8') as f:    f.write(reporte_completo)print("✓ Reporte final guardado en: seccion5_reporte_final.txt")# Guardar resultados completos de Sección 5seccion5_results = {    'task18': task18_results if 'task18_results' in locals() else None,    'task19': task19_results if 'task19_results' in locals() else None,    'task20': {        'reporte': reporte_completo,        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')    }}with open('seccion5_complete_results.pkl', 'wb') as f:    pickle.dump(seccion5_results, f)print("✓ Resultados completos guardados en: seccion5_complete_results.pkl")print("\n" + "=" * 80)print("🎉 SECCIÓN 5 COMPLETADA EXITOSAMENTE 🎉")print("=" * 80)print("\nArchivos generados:")print("  • tarea18_supervised_vs_unsupervised.png")print("  • tarea18_confusion_matrix_clusters.png")print("  • task18_results.pkl")print("  • tarea19_comparison_all_improvements.png")print("  • tarea19_best_model_confusion_matrix.png")print("  • task19_results.pkl")print("  • seccion5_reporte_final.txt")print("  • seccion5_complete_results.pkl")print("\n" + "=" * 80)print("Todas las tareas (18, 19, 20) han sido completadas")print("El proyecto está listo para presentación y entrega")print("=" * 80)